# BLU02 - Exercises Notebook

In [1]:
import hashlib # for grading

import os
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

## 1 Read the Programs data (graded)

In this first exercise, we aim to create a single dataframe, combining all programs from all seasons.

With a caveat though: **we want to include seasons after 1950**.

In [2]:
def read_season(folder_path, file_name):
    path = os.path.join(folder_path, file_name)
    return pd.read_csv(path)

def read_programs(folder_path):
    files = os.listdir(folder_path)
    # Create a list with the name of all files containing programs from
    # 1950 inclusive and onwards (just the filename, no complete path.)
    # files_after_1950: List[str] = ...
    # YOUR CODE HERE
    files_after_1950 = filter(lambda file: int(file[:4]) >= 1950, files)

    # Create a list with the dataframes
    # seasons: List[pd.DataFrame] = ...
    # YOUR CODE HERE
    seasons = [read_season(folder_path, file) for file in files_after_1950]
    
    # Use pd.concat to create a single dataframe.
    # programs: pd.DataFrame = ...
    # YOUR CODE HERE
    programs = pd.concat(seasons)
    
    # Drop the column GUID.
    # programs = ...
    # YOUR CODE HERE
    programs = programs.drop('GUID', axis=1)
    
    ## Remove Duplicated lines.
    # YOUR CODE HERE
    programs = programs.drop_duplicates()
    
    # Set the index to be the column ProgramID
    # YOUR CODE HERE
    programs = programs.set_index('ProgramID')
    
    return programs

programs = read_programs(os.path.join('data', 'programs'))

In [3]:
assert programs['Season'].min() == '1950-51'
assert programs['Season'].max() == '2016-17'
assert programs.index.name == 'ProgramID'
assert programs.shape == (7341, 2)

## 2 Read the Concerts data (graded)

Although we list all transformations step-by-step for the sake of clarity, we expect you to use method chaining.

In [4]:
def convert_date(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    df['Date'] = df['Date'].astype('datetime64')
    return df

def convert_time(df):
    df = df.copy()
    df['Time'] = pd.to_datetime(df['Time'], format='%I:%M%p').dt.date
    df['Time'] = df['Time'].astype('datetime64')
    return df

def make_concerts(file_path):
    # Read concerts data and drop the GUID column.
    # concerts: pd.DataFrame = ...
    # YOUR CODE HERE
    
    # Remember to_datetime? We need here. We need to parse the columns Date and 
    # Time. Use pd.to_datetime(...).dt.date for the Date and pd_to_datetime(..., 
    # format=%I:%M%p).dt.time for the Time.
    # YOUR CODE HERE
    
    
    ## Remove Duplicated lines.
    # YOUR CODE HERE
    
    concerts = pd.read_csv(file_path)
    
    concerts = (concerts.drop('GUID', axis=1)
                .pipe(convert_date)
                .pipe(convert_time)
                .drop_duplicates())
    
    return concerts

concerts = make_concerts(os.path.join('data','concerts.csv'))

In [5]:
concerts.head()

,ProgramID,ConcertID,EventType,Location,Venue,Date,Time
0,3853,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07,1900-01-01
1,5178,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-02-18,1900-01-01
2,10785,0,Special,"Manhattan, NY",Apollo Rooms,1843-04-07,1900-01-01
3,5887,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-04-22,1900-01-01
4,305,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-11-18,NaT


In [6]:
assert concerts.shape == (21582, 7)
assert concerts.Date.min() == datetime.date(1842, 12, 7)
assert concerts.Date.max() == datetime.date(2017, 7, 7)
assert set(concerts.columns) == set([
    'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue', 'Date', 'Time'
])

## 3 Combine Programs and Concerts data (graded)

Let's combine both dataframes into a single dataset, using an inner join.

In [7]:
# Remember that you want to join on the index of one of the dataframes.
# Join only the concerts with valid ProgramIDs
# nyp = ...
# YOUR CODE HERE

nyp = programs.merge(concerts, on='ProgramID')

In [8]:
nyp.head()

,ProgramID,Orchestra,Season,ConcertID,EventType,Location,Venue,Date,Time
0,13889,New York Philharmonic,2016-17,0,Non-Subscription,"Manhattan, NY",David Geffen Hall,2016-09-13,1900-01-01
1,13889,New York Philharmonic,2016-17,1,Non-Subscription,"Manhattan, NY",David Geffen Hall,2016-09-14,1900-01-01
2,13889,New York Philharmonic,2016-17,2,Non-Subscription,"Manhattan, NY",David Geffen Hall,2016-09-15,1900-01-01
3,13890,New York Philharmonic,2016-17,0,Non-Subscription,"Manhattan, NY",David Geffen Hall,2016-09-16,1900-01-01
4,14068,New York Philharmonic,2016-17,0,Non-Subscription,"Manhattan, NY",David Geffen Hall,2016-09-17,1900-01-01


In [9]:
assert nyp.shape == (13254, 9)
assert set(nyp.columns) == set([
    'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue',
    'Date', 'Time', 'Orchestra', 'Season'
])

## 4 Read Works and Soloists data (graded)

We will read the two remaining pieces of data. 

Again, albeit the step-by-step description, we encourage you to use method chaining.

In [10]:
def read_works(file_path):
    # Read the works data.
    # works: pd.DataFrame = ...
    # YOUR CODE HERE
    
    # Remove the Intervals (attention to the values in the isInterval column).
    # works: pd.DataFrame = ...
    # YOUR CODE HERE

    # Select the columns ProgramID, WorkID, ComposerName, WorkTitle, Movement and ConductorName.
    # YOUR CODE HERE

    ## Remove Duplicated lines.
    # YOUR CODE HERE

    works = (pd.read_csv(file_path)
            .pipe(lambda df: df[(df.isInterval == False) & (df.Interval.isnull())])
            .pipe(lambda df: df[['ProgramID', 'WorkID', 'ComposerName', 'WorkTitle', 'Movement', 'ConductorName']])
            .drop_duplicates())

    return works


def read_soloists(file_path):
    # Read the soloists data and drop GUID and MovementID Columns
    # YOUR CODE HERE

    ## Remove Duplicated lines.
    # YOUR CODE HERE
    soloists = (pd.read_csv(file_path)
                .drop('GUID', axis=1)
                .drop('MovementID', axis=1)
                .drop_duplicates())
             
    return soloists


works = read_works('data/works.csv')
soloists = read_soloists('data/soloists.csv')

In [11]:
assert works.shape == (71065, 6)
assert set(works.columns) == set([
    'ProgramID', 'WorkID', 'ComposerName', 'WorkTitle', 'Movement', 'ConductorName'
])

assert soloists.shape == (50292, 5)
assert set(soloists.columns) == set([
   'ProgramID', 'WorkID', 'SoloistName', 'SoloistInstrument', 'SoloistRole'
])

## 5 Combine Works and Soloists (graded)

Like we did for Programs and Concerts, now we combine Works and Soloists.

In [12]:
# Combine both dataframes, again using an inner type of join. An work is identified by the pair
# ProgramId, WorkID
# works_and_soloists : pd.DataFrame = ....
# YOUR CODE HERE

works_and_soloists = works.merge(soloists, on=['ProgramID', 'WorkID'])

In [13]:
assert works_and_soloists.shape == (64698, 9)
assert set(works_and_soloists.columns) == set(
    [
        'ProgramID', 'WorkID', 'ComposerName', 'WorkTitle', 'Movement',
        'ConductorName', 'SoloistName', 'SoloistInstrument', 'SoloistRole'
    ]
)

## 6 Combine everything (graded)

The final goal here is to create a single dataframe.

In [14]:
# Combine works_and_soloists and nyp into a single dataframe.
# nyp_merged = ...
# YOUR CODE HERE
nyp_merged = works_and_soloists.merge(nyp, on='ProgramID')

In [15]:
assert nyp_merged.shape == (72185, 17)
assert set(nyp_merged.columns) == set(
    [
       'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue', 'Date',
       'Time', 'Orchestra', 'Season', 'WorkID', 'ComposerName', 'WorkTitle',
       'Movement', 'ConductorName', 'SoloistName', 'SoloistInstrument',
       'SoloistRole'
    ]
)

## 7 Final transformations (graded)

Now, we perform the train-test split.

We also perform some final transformations on both datasets:

* Include some date features: Year, Month, Hour, Minute, Day and Weekday
* Create a new feature, ComposerLastName from ComposerName column. 
* Filter out rows that have a location that appear is less than 10 times in the DataFrame.
* Drop ProgramID, ConcertID, WorkID, Date and Season

In [16]:
nyp_merged.head()

,ProgramID,WorkID,ComposerName,WorkTitle,Movement,ConductorName,SoloistName,SoloistInstrument,SoloistRole,Orchestra,Season,ConcertID,EventType,Location,Venue,Date,Time
0,316,2579,"Brahms, Johannes","CONCERTO, PIANO, NO. 2, B FLAT MAJOR, OP.83",NaN,"Walter, Bruno","Hess, Myra",Piano,S,New York Philharmonic,1950-51,0,Subscription Season,"Manhattan, NY",Carnegie Hall,1951-02-08,1900-01-01
1,316,2579,"Brahms, Johannes","CONCERTO, PIANO, NO. 2, B FLAT MAJOR, OP.83",NaN,"Walter, Bruno","Hess, Myra",Piano,S,New York Philharmonic,1950-51,1,Subscription Season,"Manhattan, NY",Carnegie Hall,1951-02-09,1900-01-01
2,1645,8033,"Saint-Saens [Saint-Saëns], Camille","DELUGE, LE, OP. 45",Prelude,"Mitropoulos, Dimitri","Corigliano, John, Sr.",Violin,A,New York Philharmonic,1950-51,0,Subscription Season,"Manhattan, NY",Carnegie Hall,1951-04-05,1900-01-01
3,1645,8033,"Saint-Saens [Saint-Saëns], Camille","DELUGE, LE, OP. 45",Prelude,"Mitropoulos, Dimitri","Corigliano, John, Sr.",Violin,A,New York Philharmonic,1950-51,1,Subscription Season,"Manhattan, NY",Carnegie Hall,1951-04-06,1900-01-01
4,1645,1518,"Saint-Saens [Saint-Saëns], Camille","CONCERTO, CELLO, NO. 1, A MINOR, OP. 33",NaN,"Mitropoulos, Dimitri","Rose, Leonard",Cello,S,New York Philharmonic,1950-51,0,Subscription Season,"Manhattan, NY",Carnegie Hall,1951-04-05,1900-01-01


In [17]:
def append_date_features(df):
    # YOUR CODE HERE
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Weekday'] = df['Date'].dt.weekday
    
    df['Hour'] = df['Time'].dt.hour
    df['Minute'] = df['Time'].dt.minute

    return df

def append_composer_last_name(df):
    # YOUR CODE HERE
    df = df.copy()
    df['ComposerLastName'] = df['ComposerName'].apply(lambda x: x.split(',')[0])
    return df

def preprocess_data(df):
    # You should follow these exact steps:
    #   1 - Include some date features: Year, Month, Hour, Minute, Day and Weekday
    #   2 - Create a new feature, ComposerLastName from ComposerName column. 
    #   3 - Filter out rows that have a location that appear is less than 10 times in the DataFrame.
    #   4 - Drop ProgramID, ConcertID, WorkID, Season, Date, Time
    #   
    # YOUR CODE HERE
    return (df.copy()
            .pipe(append_date_features)
            .pipe(append_composer_last_name)
            #.pipe(filter_locations)
            .groupby('Location').filter(lambda x: x.shape[0] >= 10)
            .drop(['ProgramID', 'ConcertID', 'WorkID', 'Season', 'Date', 'Time'], axis=1)
           )
    
nyp_preprossed = preprocess_data(nyp_merged)

In [18]:
assert nyp_preprossed.shape == (71659, 18)
assert set(nyp_preprossed.columns) == {
       'EventType', 'Location', 'Venue', 'Orchestra',
       'ComposerName', 'WorkTitle', 'Movement', 'ConductorName', 'SoloistName',
       'SoloistInstrument', 'SoloistRole', 'Year', 'Month', 'Day', 'Hour',
       'Minute', 'Weekday', 'ComposerLastName'
}
assert nyp_preprossed.groupby('Location').size().min() == 10
assert nyp_preprossed.ComposerLastName.value_counts().loc['Mozart'] == 3333
assert nyp_preprossed.ComposerLastName.value_counts().loc['Gershwin'] == 2127
assert nyp_preprossed.ComposerLastName.nunique() == 1241

# The house prices dataset

A dataset containing several characteristics of several houses and their sell price 

* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* OverallQual: Rates the overall material and finish of the house
       10  Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
* OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
* MasVnrArea: Masonry veneer area in square feet
* BsmtFinSF1: Type 1 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* BedroomAbvGr: Bedrooms above grade (does NOT include basement bedrooms)
* KitchenAbvGr: Kitchens above grade
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Fireplaces: Number of fireplaces
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* MiscVal: $Value of miscellaneous feature 
* SellingDate: Date when the house was sold
* BuildingDate: Date when the house was built
* RemodAddDate: Remodel date (same as construction date if no remodeling or additions)
* SalePrice: The house price at the selling date (our target variable)

In [19]:
def house_price_dataset():
    return pd.read_csv(
    'data/housePrices.csv', 
        parse_dates=[
            'SellingDate',
            'BuildingDate',
            'RemodAddDate'
        ]
    )

dataset = house_price_dataset()
dataset_train, dataset_test = train_test_split(dataset, random_state=0)
X_train = dataset_train.drop(columns='SalePrice')
y_train = dataset_train.SalePrice
X_test = dataset_test.drop(columns='SalePrice')
y_test = dataset_test.SalePrice

## 8 Build a DateTransformer transformer (graded)

There's a simple transformer that can be useful, from times to times, when modeling.

What we want is to build a transformer that transform dates into timedeltas.

Usually when you have features that are Dates you compute a time delta between the feature and a given refence date.

e.g Imagine that your clients have a loyalty period that ends at a given date. When your model is doing some predictions, one of the features that you can use is the number of days until the end of the loyalty period. i.e the date when the loyalty ends minus the date when your model is running. 

In the house prices dataset, the selling date will be the reference data, since we want to predict the house price at the selling date. Then we need to convert the other date using our transformer

In [20]:
class DateTransformer(BaseEstimator, TransformerMixin):
    # Implement the __init__ method.
    # Our DateTransformer must be able to receive two parameters: 
    # datetime_cols: a list, that contains the datetime cols that should be converted
    # ref_date_col - indicates the name of the column that should be used as reference date,
    # YOUR CODE HERE
        
    # There's no need for a fit method in this case, it does nothing.
    # We should be able to call fit without any explicit parameters.
    # Meaning: we should be able to call transformer.fit().
    # YOUR CODE HERE

    # Transform should transform all datetime columns into the difference in days to the reference date.
    # The reference date column should be dropped. 
    # YOUR CODE HERE

    def __init__(self, datetime_cols, ref_date_col):
        self.datetime_cols =  datetime_cols
        self.ref_date_col =  ref_date_col
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = pd.DataFrame(X)
        
        for datetime_col in self.datetime_cols:
            df[datetime_col] = (df[datetime_col].astype('datetime64[ns]') - df[self.ref_date_col].astype('datetime64[ns]')).astype("timedelta64[D]").astype('int64')
        
        df = df.drop(self.ref_date_col, axis=1)
        return df

In [21]:
X_train_trasformed = DateTransformer(
    datetime_cols=['BuildingDate', 'RemodAddDate'], 
    ref_date_col='SellingDate'
).fit_transform(X_train)
assert X_train_trasformed.BuildingDate.min() == -49008
assert X_train_trasformed.BuildingDate.max() == -1
assert 'SellingDate' not in X_train_trasformed.columns
assert X_train_trasformed.dtypes.BuildingDate == np.dtype('int64')
assert X_train_trasformed.dtypes.RemodAddDate == np.dtype('int64')

## 9 Building the pipeline (graded)

Finally, we want to use the two transformers together and run a linear regression on top.

We want to Covert hour dates to time deltas relative to the Selling Date.

We want to scale all features to the same range, using `sklearn.preprocessing.StandardScaler()`.

We want to estimate the SellingPrice using a Liner Regression.

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.



In [23]:
# Create a pipeline including:
#   1 - 'date_converter', DateTransformer(['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')
#   2 - 'standard_scaller', StandardScaler() with the default parameters
#   3 - 'model', LinearRegression
# YOUR CODE HERE

pipeline = Pipeline([('date_converter', DateTransformer(datetime_cols=['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')),
                     ('standard_scaller', StandardScaler()),
                     ('model', LinearRegression())
                    ])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print('MAE: {}'.format(mae))

MAE: 248535026.46842527


In [24]:
assert type(pipeline) == Pipeline
assert type(pipeline.named_steps['date_converter']) == DateTransformer
assert type(pipeline.named_steps['standard_scaller']) == StandardScaler
assert pipeline.named_steps['date_converter'].get_params()['ref_date_col'] == 'SellingDate'
assert set(
    pipeline.named_steps['date_converter'].get_params()['datetime_cols']
) == {'BuildingDate', 'RemodAddDate'}
assert type(pipeline.named_steps['model']) == LinearRegression 

Exercises complete, congratulations! You are about to become a certified data wrangler.